## Task 1

In [1]:
from pymongo import MongoClient
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import csv


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Chris\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#select the mongo client 
client = MongoClient = MongoClient(port = 27017)
#select the mongoDB database
db = client["COMP3210-ASSIGN1"]
#select the mongodb tweets dataset
tweets = db["10000 tweets"]

In [4]:
print(tweets)
type(tweets)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'COMP3210-ASSIGN1'), '10000 tweets')


pymongo.collection.Collection

In [28]:
#function remove all stop words
def remove_stopwords(words):
    result =[]
    for w in words:
        if w not in stopwords.words("english"):
            result.append(w)
    return result
#function to develop stem words through porter stemmer
def stem_words(words):
    ps = PorterStemmer()
    result =[ps.stem(w) for w in words]
    return result

#function to find keywords within text uses both remove stop words and stem words
def find_keywords_in_text(words):
    text = re.sub(r"[^\w]", " ", words)
    words = word_tokenize(text)
    words=remove_stopwords(words)
    words=stem_words(words)
    keywords=nltk.pos_tag(words)
    return keywords

#converting keywords into a common speator to text
def keywords_to_csv(keywords):
    csv:str= ""
    for k in keywords:
        column: str = k[0]+","+k[1]
        csv = csv + column + "\n"
    return csv


In [29]:
#loops through each tweet, and extracts keywords
# updates the database with the keywords in CSV format
# 1 for using and 0 for not using
for t in tweets.find({},{"id":1, "body":1, "_id":0}):
    keywords = find_keywords_in_text(t["body"])
    csv=keywords_to_csv(keywords)
    tweets.update_many({"id":t["id"]}, {"$set":{"keyword":csv}})

## Task 2

In [15]:
#Remove URLS from https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
#Remove Emojis https://www.educative.io/edpresso/how-to-remove-emoji-from-the-text-in-python
# in command prompt pip install clean-text
# must make all words lowercase so we do not make the problem of eg best and BEST not being the same
# This url helped to complete the pre-procesing steps for twitter : https://www3.tuhh.de/sts/hoou/data-quality-explored/3-2-simple-transf.html
from unidecode import unidecode # helps to decode more efficently with minimal bias

def remove_urls(txt):
    # txt = re.sub(r"http\S+", "", txt) # flags=re.MULTILINE)
    txt = re.sub(r'https?://[^ ]+', '', txt)
    return(txt)
def remove_user(txt):
    txt = re.sub(r'@[^ ]+', '', txt)
    return txt
def remove_hashtag(txt):
    txt = re.sub(r'#', '', txt)
    return txt
#character normalisation
def char_norm(txt):
    txt = re.sub(r'([A-Za-z])\1{2,}', r'\1', txt)
    return txt
#removing punctuation, special characters and numbers
def remove_pun_sc_nums(txt):
    txt = re.sub(r' 0 ', 'zero', txt)
    txt = re.sub(r'[^A-Za-z ]', '', txt)
    return txt
#Remove Emojis https://www.educative.io/edpresso/how-to-remove-emoji-from-the-text-in-python
# in command prompt pip install clean-text
from cleantext import clean
def clean_data(txt):
    return remove_pun_sc_nums(char_norm(remove_hashtag(remove_user(remove_urls(clean(txt.lower(), no_emoji=True))))))

In [86]:
#add all posts within a string variable so then I can use my clean_data function, word tokenize and remove stop words
body_text = ''
for t in tweets.find():
    body_text+=t["body"]
tok_body_text = remove_stopwords(word_tokenize(clean_data(body_text)))

In [88]:
#print out into a text file to use for my task 2 mapreduce function
with open('Task2_body_cleaned.txt', 'w', encoding='utf-8') as f:
    for t in tok_body_text:
        f.write(t)
        f.write("\n")

## Task 3

In [207]:
#Input file for task 3
# aus_cities  = ('Sydney' or 'Brisbane' or 'Mebourne'or 'Perth'or 'Adelaide'or 'Hobart'or 'Canberra' or 'Darwin')
# Due to getting not all cities from australia when filtering twitterzone and display name. I create a list which if contains the city will print into text file
aus_cities  = ['Sydney','Brisbane','Melbourne','Perth','Adelaide','Hobart','Canberra','Darwin']
with open('Task3_cities.txt', 'w', encoding='utf-8') as f:
    for t in tweets.find({"actor.location.displayName": {'$regex' : 'Australia', "$options" : "i"}}):
        if(t["actor"]["twitterTimeZone"] != None and t["actor"]["twitterTimeZone"] in aus_cities):
            f.write(t["actor"]["twitterTimeZone"])
            f.write("\n")

## Task 4 - Merge sort

In [9]:
#grab ids and splits the extra stuff such as the twitter.com part to only get the ids.
#used for both merge and bucket sort
with open('task4_input.txt', 'w', encoding='utf-8') as f:
    for t in tweets.find():
        f.write(t["object"]["id"].split(':')[2])
        f.write("\n")

## Task 4 - Bucket

In [10]:
#preprocesing is found within Task 4- Merge section


In [4]:
#determining all twitter id first digit numbers
leng = [] 
for t in tweets.find():
    leng.append(t["object"]["id"].split(':')[2][0])
    
leng = set(leng)
print(leng)



{'2', '7', '3', '4', '5', '6'}


In [67]:
##Checking the ratio of digits with particular first digit numbers within the twitter ids
two = 0
three = 0
four = 0
five = 0
six = 0
seven = 0
eight = 0
total = 0
for t in tweets.find():
    if t["object"]["id"].split(':')[2][0] == '2':
        two+=1
    if t["object"]["id"].split(':')[2][0] == '3':
        three+=1
    if t["object"]["id"].split(':')[2][0] == '4':
        four+=1
    if t["object"]["id"].split(':')[2][0] == '5':
        five+=1
    if t["object"]["id"].split(':')[2][0] == '6':
        six+=1
    if t["object"]["id"].split(':')[2][0] == '7':
        seven+=1
    total+=1
print({"Two" :two, "Three" :three, "Four" :four, "Five" :five, "Six" :six, "Seven": seven, "Eight": eight})

{'Two': 1, 'Three': 2, 'Four': 6, 'Five': 11, 'Six': 65, 'Seven': 9915, 'Eight': 0}


In [114]:
# print({"Five" :(five/total)*100, "Six" :(six/total)*100, "Seven": (seven/total)*100, "Eight": (eight/total)*100, "Nine":(nine/total)*100, "Ten":(ten/total)*100, "Eighteen": (eighteen/total)*100})

{'Five': 0.04, 'Six': 0.11, 'Seven': 0.54, 'Eight': 18.75, 'Nine': 34.72, 'Ten': 42.21, 'Eighteen': 3.63}


## Task 5

In [16]:
#developed a dictionary where keys is twitter ids and values are twitter post text
# I did this as I found it an easy approach to using my cleaning data for my post text (which is find necessary)
ids_text_dic = {}
for i in tweets.find():
    ids_text_dic.update({i["id"].split(':')[2] : clean_data(i["body"])})

In [21]:
#did this to determine how many times health occurances in each post, based on results tweets are found mostly once in a document which contains it.
count_health = []
for i in ids_text_dic.values():
    if 'health' in i:
        count_health.append(i.count("health"))
print(set(count_health)
)


[1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1]


In [30]:

# preprocessing my dictionary data into a text file where I used this symbol : so I can split both parts into an array within my mapreduce function
with open('Task5_ids_body.txt', 'w', encoding='utf-8') as f:
    for ids ,body in ids_text_dic.items():
        f.write(ids)
        f.write(':')
        f.write(body)
        f.write("\n")